# ReAct Agent

Adapted from https://github.com/opendatahub-io/llama-stack-demos/blob/main/demos/rag_agentic/notebooks/Level3_advance_agentic_with_Prompt_Chaining_ReAct.ipynb

In [9]:
import rich
import uuid
from llama_stack_client import Agent, LlamaStackClient, AgentEventLogger
from llama_stack_client.lib.agents.client_tool import client_tool
from llama_stack_client.lib.agents.event_logger import EventLogger
from llama_stack_client.lib.agents.react.agent import ReActAgent
from llama_stack_client.lib.agents.react.tool_parser import ReActOutput

In [10]:
# this may be missing from pyproject.toml, add it if you need it
#!uv pip install geocoder

In [11]:
@client_tool
def get_location(query: str = "location"):
    """
    Provide the location upon request.

    :param query: The query from user
    :returns: Information about user location
    """
    import geocoder
    
    try:
        g = geocoder.ip('me')
        if g.ok:
            return f"Your current location is: {g.city}, {g.state}, {g.country}" # can be modified to return latitude and longitude if needed
        else:
            return "Unable to determine your location"
    except Exception as e:
        return f"Error getting location: {str(e)}"

In [12]:
client = LlamaStackClient(
    base_url="http://localhost:8321",
)

In [13]:
models = client.models.list()
rich.print(models)

INFO:httpx:HTTP Request: GET http://localhost:8321/v1/models "HTTP/1.1 200 OK"


[
    Model(
        identifier='all-MiniLM-L6-v2',
        metadata={'embedding_dimension': 384.0},
        api_model_type='embedding',
        provider_id='ollama',
        type='model',
        provider_resource_id='all-minilm:latest',
        model_type='embedding'
    ),
    Model(
        identifier='llama3.2:3b-instruct-fp16',
        metadata={},
        api_model_type='llm',
        provider_id='ollama',
        type='model',
        provider_resource_id='llama3.2:3b-instruct-fp16',
        model_type='llm'
    )
]

In [16]:
toolgroups = client.toolgroups.list()
rich.print(toolgroups)

INFO:httpx:HTTP Request: GET http://localhost:8321/v1/toolgroups "HTTP/1.1 200 OK"


[
    ToolGroup(
        identifier='builtin::rag',
        provider_id='rag-runtime',
        type='tool_group',
        args=None,
        mcp_endpoint=None,
        provider_resource_id='builtin::rag'
    ),
    ToolGroup(
        identifier='builtin::websearch',
        provider_id='tavily-search',
        type='tool_group',
        args=None,
        mcp_endpoint=None,
        provider_resource_id='builtin::websearch'
    ),
    ToolGroup(
        identifier='builtin::wolfram_alpha',
        provider_id='wolfram-alpha',
        type='tool_group',
        args=None,
        mcp_endpoint=None,
        provider_resource_id='builtin::wolfram_alpha'
    ),
    ToolGroup(
        identifier='mcp::filesystem',
        provider_id='model-context-protocol',
        type='tool_group',
        args=None,
        mcp_endpoint=McpEndpoint(uri='http://localhost:8000/sse'),
        provider_resource_id='mcp::filesystem'
    )
]

In [18]:
tools = client.tools.list()
rich.print(tools)

INFO:httpx:HTTP Request: GET http://localhost:8321/v1/tools "HTTP/1.1 500 Internal Server Error"
INFO:llama_stack_client._base_client:Retrying request to /v1/tools in 0.465991 seconds
INFO:httpx:HTTP Request: GET http://localhost:8321/v1/tools "HTTP/1.1 500 Internal Server Error"
INFO:llama_stack_client._base_client:Retrying request to /v1/tools in 0.855234 seconds
INFO:httpx:HTTP Request: GET http://localhost:8321/v1/tools "HTTP/1.1 500 Internal Server Error"


InternalServerError: Error code: 500 - {'detail': 'Internal server error: An unexpected error occurred.'}

## Baseline

In [19]:
client.toolgroups.register(
    toolgroup_id="builtin::websearch",
    provider_id="tavily-search",
)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/toolgroups "HTTP/1.1 200 OK"


In [20]:
toolgroups = client.toolgroups.list()
rich.print(toolgroups)

INFO:httpx:HTTP Request: GET http://localhost:8321/v1/toolgroups "HTTP/1.1 200 OK"


[
    ToolGroup(
        identifier='builtin::rag',
        provider_id='rag-runtime',
        type='tool_group',
        args=None,
        mcp_endpoint=None,
        provider_resource_id='builtin::rag'
    ),
    ToolGroup(
        identifier='builtin::websearch',
        provider_id='tavily-search',
        type='tool_group',
        args=None,
        mcp_endpoint=None,
        provider_resource_id='builtin::websearch'
    ),
    ToolGroup(
        identifier='builtin::wolfram_alpha',
        provider_id='wolfram-alpha',
        type='tool_group',
        args=None,
        mcp_endpoint=None,
        provider_resource_id='builtin::wolfram_alpha'
    ),
    ToolGroup(
        identifier='mcp::filesystem',
        provider_id='model-context-protocol',
        type='tool_group',
        args=None,
        mcp_endpoint=McpEndpoint(uri='http://localhost:8000/sse'),
        provider_resource_id='mcp::filesystem'
    )
]

In [21]:
agent = Agent(
    client,
    model="llama3.2:3b-instruct-fp16",
    instructions="You are a helpful assistant.",
    tools=["builtin::websearch"],
)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8321/v1/tools?toolgroup_id=builtin%3A%3Awebsearch "HTTP/1.1 200 OK"


In [22]:
response = agent.create_turn(
    messages=[{"role": "user", "content": "Are you able to search the web?"}],
    session_id=agent.create_session(f"tool-check-{uuid.uuid4()}"),
    stream=False,
)
rich.print(response)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/cd3f947d-3683-4945-a91b-2091edaa2082/session "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/cd3f947d-3683-4945-a91b-2091edaa2082/session/86d1b095-cbea-47ef-8729-b675392a138b/turn "HTTP/1.1 200 OK"


Turn(
    input_messages=[UserMessage(content='Are you able to search the web?', role='user', context=None)],
    output_message=CompletionMessage(
        content='Yes, I can search the web for information on a wide range of topics. What would you like me to 
look up?',
        role='assistant',
        stop_reason='end_of_turn',
        tool_calls=[]
    ),
    session_id='86d1b095-cbea-47ef-8729-b675392a138b',
    started_at=datetime.datetime(2025, 6, 6, 15, 20, 4, 383362, tzinfo=TzInfo(UTC)),
    steps=[
        InferenceStep(
            api_model_response=CompletionMessage(
                content='Yes, I can search the web for information on a wide range of topics. What would you like 
me to look up?',
                role='assistant',
                stop_reason='end_of_turn',
                tool_calls=[]
            ),
            step_id='d72e8565-a336-4205-a380-4851d6b4c0c3',
            step_type='inference',
            turn_id='4308a118-845e-41ae-a687-d50f58d10651',
            completed_at=datetime.datetime(2025, 6, 6, 15, 20, 13, 243094, tzinfo=TzInfo(UTC)),
            started_at=datetime.datetime(2025, 6, 6, 15, 20, 4, 383654, tzinfo=TzInfo(UTC))
        )
    ],
    turn_id='4308a118-845e-41ae-a687-d50f58d10651',
    completed_at=datetime.datetime(2025, 6, 6, 15, 20, 13, 254295, tzinfo=TzInfo(UTC)),
    output_attachments=[]
)

In [23]:
agent = Agent(
    client, 
    model="llama3.2:3b-instruct-fp16",
    instructions="You are a helpful assistant. Use websearch tool to help answer questions.",
    tools=["builtin::websearch"],
)
user_prompts = [
    # "Are there any weather-related risks in my area?",
    "Are there any weather-related risks in my area that could disrupt network connectivity or system availability?",

]
session_id = agent.create_session(f"test-session-{uuid.uuid4()}")
for prompt in user_prompts:
    rich.print(f"User> {prompt}")

    response = agent.create_turn(
        messages=[{"role": "user", "content": prompt}],
        session_id=session_id,
        stream=True,
    )
    for log in AgentEventLogger().log(response):
        log.print()

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8321/v1/tools?toolgroup_id=builtin%3A%3Awebsearch "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/d51c1139-c3f4-4029-8ccd-8a0fc402923b/session "HTTP/1.1 200 OK"


User> Are there any weather-related risks in my area that could disrupt network connectivity or system 
availability?

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/d51c1139-c3f4-4029-8ccd-8a0fc402923b/session/12e6dc3c-6170-4df6-a8b7-a15721988109/turn "HTTP/1.1 200 OK"


inference> brave_search.call(query="weather-related risks near me")
tool_execution> Tool:brave_search Args:{'query': 'weather-related risks near me'}
tool_execution> Tool:brave_search Response:{"query": "weather-related risks near me", "top_k": [{"title": "Weather in near me", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'Phumi Near Pisei', 'region': 'Koh Kong', 'country': 'Cambodia', 'lat': 11.1333, 'lon': 103.5833, 'tz_id': 'Asia/Phnom_Penh', 'localtime_epoch': 1749223233, 'localtime': '2025-06-06 22:20'}, 'current': {'last_updated_epoch': 1749222900, 'last_updated': '2025-06-06 22:15', 'temp_c': 26.3, 'temp_f': 79.3, 'is_day': 0, 'condition': {'text': 'Patchy light rain in area with thunder', 'icon': '//cdn.weatherapi.com/weather/64x64/night/386.png', 'code': 1273}, 'wind_mph': 4.3, 'wind_kph': 6.8, 'wind_degree': 258, 'wind_dir': 'WSW', 'pressure_mb': 1009.0, 'pressure_in': 29.81, 'precip_mm': 0.62, 'precip_in': 0.02, 'humidity': 91, 'cloud': 89, 'feelsli

## Prompt Chaining

In [24]:
agent = Agent(
    client,
    model="llama3.2:3b-instruct-fp16",
    instructions="""You are a helpful assistant. 
    When a user asks about their location, use the get_location tool. 
    When searching for nearby places, use the websearch tool.
    """,
    tools=[get_location, "builtin::websearch"],
)
user_prompts = [
    "Where am I?",
    "Are there any weather-related risks in my area?",
]
session_id = agent.create_session(f"prompt-chaining-session-{uuid.uuid4()}")
for prompt in user_prompts:
    rich.print(f"User> {prompt}")
    response = agent.create_turn(
        messages=[{"role": "user", "content": prompt}],
        session_id=session_id,
    )
    for log in EventLogger().log(response):
        log.print()

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8321/v1/tools?toolgroup_id=builtin%3A%3Awebsearch "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/5af7d2e9-4f88-4cc0-b29c-232466a0481e/session "HTTP/1.1 200 OK"


User> Where am I?

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/5af7d2e9-4f88-4cc0-b29c-232466a0481e/session/76b3df00-3bd9-44ea-9452-7a23063d018d/turn "HTTP/1.1 200 OK"


inference> [get_location(query="my current location")]


INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/5af7d2e9-4f88-4cc0-b29c-232466a0481e/session/76b3df00-3bd9-44ea-9452-7a23063d018d/turn/ad1a797f-968e-4da7-995f-138a594dade3/resume "HTTP/1.1 200 OK"


...

User> Are there any weather-related risks in my area?

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/5af7d2e9-4f88-4cc0-b29c-232466a0481e/session/76b3df00-3bd9-44ea-9452-7a23063d018d/turn "HTTP/1.1 200 OK"


inference> [get_location(query="weather risks near me")]


INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/5af7d2e9-4f88-4cc0-b29c-232466a0481e/session/76b3df00-3bd9-44ea-9452-7a23063d018d/turn/aa68882f-1863-4974-966f-9fbabe7b4cac/resume "HTTP/1.1 200 OK"


tool_execution> Tool:get_location Args:{'query': 'weather risks near me'}
tool_execution> Tool:get_location Response:Error when running tool: No module named 'geocoder'
inference> import geocoder

location = geocoder.ip('me')
print(location.weather)<|python_tag|>import geocoder

location = geocoder.ip('me')
print(location.weather)


### ReAct Agent 

This section demonstrates the ReAct (Reasoning and Acting) framework in action.

For example, when asked "Are there any weather-related risks in my area that could disrupt network connectivity or system availability?", the agent:
1. **Reasons** it needs location information first
2. **Acts** by calling the `get_location` client tool
3. **Observes** the location result
4. **Reasons** about the next step
5. **Acts** by calling the `websearch` tool with observed location
6. **Observes** and processes the search results

Unlike prompt chaining which follows fixed steps, ReAct dynamically breaks down tasks and adapts its approach based on the results of each step. This makes it more flexible and capable of handling complex, real-world queries effectively.

In [25]:
agent = ReActAgent(
    client=client,
    model="llama3.2:3b-instruct-fp16",
    instructions="You are a helpful assistant. Use the tools at your disposal to answer any questions.",
    tools=[get_location, "builtin::websearch"],
    response_format={
        "type": "json_schema",
        "json_schema": ReActOutput.model_json_schema(),
    },
)
user_prompts = [
    "Are there any weather-related risks in my area?",
]
session_id = agent.create_session(f"web-session-{uuid.uuid4()}")
for prompt in user_prompts:
    rich.print(f"User> {prompt}")
    response = agent.create_turn(
        messages=[{"role": "user", "content": prompt}],
        session_id=session_id,
    )
    for log in EventLogger().log(response):
        log.print()

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8321/v1/tools?toolgroup_id=builtin%3A%3Awebsearch "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/e4137a24-9b00-4557-b813-f029bff299fe/session "HTTP/1.1 200 OK"


User> Are there any weather-related risks in my area?

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/e4137a24-9b00-4557-b813-f029bff299fe/session/714e5cfd-0bb5-4dfb-9778-ab10dbe685b1/turn "HTTP/1.1 200 OK"


inference> 500: Internal server error: An unexpected error occurred.
